In [18]:
import pandas as pd
import numpy as np
from utils import data
import openai
import os
from absl import app, flags, logging
import time
language = 'itl'
openai.api_key = os.getenv("OPENAI_API_KEY")

train_path = f"data/{language}/{language}.train"
dev_path = f"data/{language}/{language}.dev"
test_path = f"data/{language}/{language}.test"

In [11]:
data=pd.read_csv(train_path, sep='\t', header=None, 
                 names=["input", "output", "tags"])
data.loc[data['tags'] == 'N']


tags_dictionary_train = {k: list(map(tuple, g[['input','output']].values)) for k, g in data.groupby('tags')}

In [12]:
with open("prompts/inflection_train_base.txt") as handle:
    base_text = handle.read()
with open("prompts/inflection_test_base.txt") as handle:
    test_text = handle.read()

In [13]:
test_data=pd.read_csv(test_path, sep='\t', header=None, 
                 names=["input", "output", "tags"])
tags_dictionary_test = {k: list(map(tuple, g[['input','output']].values)) for k, g in test_data.groupby('tags')}

In [14]:
all_tags = []
for i in tags_dictionary_train:
    all_tags.append(i)
for i in tags_dictionary_test:
    all_tags.append(i)
all_tags = list(set(all_tags))
len(all_tags)

300

In [15]:
tags_dictionary_train["ADJ;FOC;SG"]

[('kɬәŋqsxi', 'kɬәŋqsxiˀin'), ('kčixlqɬ', 'kčixlqɬˀin')]

In [16]:
import random
def get_prompt(tag, train, test):
    prompt = []
    try:
        train_list = train[tag]    
    except Exception:
        return 'cant index'
    
    if len(train_list) >= 10:
        train_list = random.sample(train_list, 10)
        
    for index, row in enumerate(train_list):
        prompt.append(base_text.format(language=language, inp=row[0],
                              tags=tag,
                              output=row[1]))
        
    prompt.append(test_text.format(language=language, inp=test[0], tags=tag))
    
    return "\n".join(prompt)

In [17]:
for index, tag in enumerate(tags_dictionary_test):
    for i in range(len(tags_dictionary_test[tag])):
        prompt = get_prompt(tag, tags_dictionary_train, tags_dictionary_test[tag][i])
        print(prompt)
        if prompt == "cant index":
            continue

       
        response = openai.Completion.create(
                            engine="text-davinci-002",
                            prompt=prompt,
                            temperature=0.7,
                            max_tokens=100,
                            top_p=1,
                            frequency_penalty=0,
                            presence_penalty=0,
                            stop=["Q"]
                        )
        gold = tags_dictionary_test[tag][i][1]
        print(gold)
        current_outputs = response["choices"][0]["text"]
        print(f"This is the current output: {current_outputs}\nThis is the gold: {gold}")
        print('\n')
        time.sleep(5)
    
    if index == 5:
        break
    

cant index
cant index
Q: Inflect the itl word 'čʼačʼ' with the morphological tags of ADJ;PL
A: čʼačʼalaχaˀn
Q: Inflect the itl word 'čʼevezlaχ' with the morphological tags of ADJ;PL
A: čʼevezlaχaˀn
Q: Inflect the itl word 'iˀɬuq' with the morphological tags of ADJ;PL
A: iˀɬuqeˀn
Q: Inflect the itl word 'kɬčl' with the morphological tags of ADJ;PL
A: kɬčleˀn
Q: Inflect the itl word 'ekʼni' with the morphological tags of ADJ;PL
A: ekʼniˀn
Q: Inflect the itl word 'ktve' with the morphological tags of ADJ;PL
A: ktvelaχeˀn
Q: Inflect the itl word 'kstʼaŋa' with the morphological tags of ADJ;PL
A:
kstʼaŋaˀn
This is the current output:  kstʼaŋalaχaˀn
This is the gold: kstʼaŋaˀn




KeyboardInterrupt: 